# Compétition Kaggle Give_Me_Some_Credit

#### Pour commencer, on récupère le fichier csv sur Kaggle

In [1]:
from bigml.api import BigML
from pandas import read_csv

df = read_csv('https://oml-data.s3.amazonaws.com/kaggle-give-me-credit-train.csv', index_col=0)

#### Connexion BigML project

In [2]:
api = BigML(project='project/5d94a454eba31d46690001d1') #Julien project/5d94a454eba31d46690001d1

#### Création de la fonction pour modifier les datasets comme on le souhaite

In [3]:
def change_dataset(dataframe):
    dataframe['IncomePerPerson'] = dataframe['MonthlyIncome'] / ( dataframe['NumberOfDependents'] + 1 )
    dataframe.loc[dataframe.age > 80, 'isOld'] = '1' 
    dataframe.loc[dataframe.age <= 80, 'isOld'] = '0'
    dataframe['MonthlyDebt'] = dataframe['MonthlyIncome'] * dataframe['DebtRatio']
    dataframe['MonthlyBalance'] = dataframe['MonthlyIncome'] - dataframe['MonthlyDebt']
    dataframe['DebtPerPerson'] = dataframe['MonthlyDebt'] / ( dataframe['NumberOfDependents'] + 1 )
    dataframe['BalancePerPerson'] = dataframe['MonthlyBalance'] / ( dataframe['NumberOfDependents'] + 1 )
    dataframe['NumberOfTime30-89DaysPastDueNotWorse'] = dataframe['NumberOfTime30-59DaysPastDueNotWorse'] + dataframe['NumberOfTime60-89DaysPastDueNotWorse']
    dataframe['NumbersOfOpen-NumberRealEstate'] = dataframe['NumberOfOpenCreditLinesAndLoans'] - dataframe['NumberRealEstateLoansOrLines']
    dataframe = dataframe.fillna(0)
    dataframe.index.names = ['Id']
    return dataframe

*Au dessus, on a les modifications sur le dataset qu'on a effectué jusqu'à ce jour avant de créer le notebook*

*__ne pas oublier d'ajouter les modifications__ faites sur le notebook __"GiveMeCredit_ContinuousImprovement"__ dans la définition __"def new_change_dataset(dataframe)"__ pour une nouvelle submission à Kaggle*

#### Application de la fonction de modification du dataset Trainfull

In [22]:
df = change_dataset(df)
df.to_csv('csv_file/train_full.csv')

## Envoi sur BigML

#### Création de la source et du dataset sur BigML via l'API

In [23]:
source = api.create_source('csv_file/train_full.csv')
api.ok(source)

origin_dataset = api.create_dataset(source)
api.ok(origin_dataset)

True

#### Création d'un modele ensemble sur la partie Train du dataset (On precise quelle est la donnée que l'on recherche)

In [24]:
ensemble = api.create_ensemble(origin_dataset , {"objective_field" : "SeriousDlqin2yrs"})

#### Modifications fichier test kaggle

##### Ce fichier doit avoir les mêmes modifications que le fichier train de kaggle qu'on a modifié au début

In [25]:
df_test_kaggle = read_csv('https://oml-data.s3.amazonaws.com/kaggle-give-me-credit-test.csv', index_col=0)

df_test_kaggle = change_dataset(df_test_kaggle)
df_test_kaggle.to_csv('csv_file/dataset_testKaggle.csv')

#### Creation d'une source / Modification du fichier test de Kaggle

In [26]:
source_test = api.create_source('csv_file/dataset_testKaggle.csv')
api.ok(source_test)

True

#### Creation d'un dataset ( = source )

In [27]:
origin_dataset_test = api.create_dataset(source_test)
api.ok(origin_dataset_test)

True

 ## Mise en forme du fichier conformément aux indications de la compétition Kaggle
 
 #### Ici, on ne veut plus que 2 colonnes :  la colonne "Id" et la colonne "1 probability"

In [28]:
batch_prediction_kaggle = api.create_batch_prediction(ensemble, origin_dataset_test,{"all_fields": True,"probabilities": True})
api.ok(batch_prediction_kaggle)
api.download_batch_prediction(batch_prediction_kaggle,filename='BatchPrediction/Prediction_Kaggle.csv')

'BatchPrediction/Prediction_Kaggle.csv'

#### Supprimer les colonnes superflues pour ne garder que les 2 souhaitées

In [30]:
import pandas as p
df_final_prediction = p.read_csv('BatchPrediction/Prediction_Kaggle.csv')
keep_col = ['Id', '1 probability']
df_final_prediction = df_final_prediction[keep_col]
df_final_prediction.rename(columns={'1 probability':'Probability'}, inplace=True)
df_final_prediction.to_csv('csv_file/FinalPrediction.csv', index=False)

#### Envoi du fichier à Kaggle pour participer à la compétition

In [31]:
import kaggle

submission_file = 'csv_file/FinalPrediction.csv'
kaggle.api.competition_submit(submission_file, "BigML ensemble", "GiveMeSomeCredit")

100%|██████████| 1.64M/1.64M [00:04<00:00, 366kB/s] 


Successfully submitted to Give Me Some Credit